In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [2]:
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9585761092214872990
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22561734528
 locality {
   bus_id: 1
 }
 incarnation: 315813598804755119
 physical_device_desc: "device: 0, name: DML, pci bus id: <undefined>"
 xla_global_id: -1]

In [3]:
# Load the dataset
data = pd.read_csv("Resultados-MegaSena.csv", encoding='latin-1')
data.head(10)

,Conc,Data,NR1,NR2,NR3,NR4,NR5,NR6
0,2601,14/06/2023,3,8,34,40,44,55
1,2600,10/06/2023,4,18,37,38,46,60
2,2599,07/06/2023,23,28,34,43,47,60
3,2598,03/06/2023,7,14,24,53,58,60
4,2597,31/05/2023,14,26,34,54,56,58
5,2596,27/05/2023,34,35,39,47,51,56
6,2595,24/05/2023,1,13,34,39,50,52
7,2594,20/05/2023,7,26,32,35,49,55
8,2593,17/05/2023,10,14,17,25,32,39
9,2592,13/05/2023,15,17,28,34,35,51


In [4]:
# separate the features and labels
data['Data'] = pd.to_datetime(data['Data'], format='%d/%m/%Y')
features = data[['Data', 'Conc']]
labels = data[['NR1', 'NR2', 'NR3', 'NR4', 'NR5', 'NR6']]

In [ ]:
# convert date column to numerical features
features['Data'] = pd.to_datetime(features['Data'])
features['Day'] = features['Data'].dt.day 
features['Month'] = features['Data'].dt.month
features['Year'] = features['Data'].dt.year
#features['DataI'] = pd.to_numeric(features['Data'])
#features['DataF'] = features['Data'].apply(lambda x: x.timestamp())
features.drop(['Data'], axis=1, inplace=True)

In [6]:
print(features)

      Conc  Day  Month  Year
0     2601   14      6  2023
1     2600   10      6  2023
2     2599    7      6  2023
3     2598    3      6  2023
4     2597   31      5  2023
...    ...  ...    ...   ...
2596     5    8      4  1996
2597     4    1      4  1996
2598     3   25      3  1996
2599     2   18      3  1996
2600     1   11      3  1996

[2601 rows x 4 columns]


In [7]:
# normalize the features (year should have more weightage than day and month)
#features['Day'] = features['Day'] / 31.0
#features['Month'] = features['Month'] / 12.0
#features['Year'] = (features['Year'] - 2000) / 20.0

In [8]:
# split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features.values, labels.values, test_size=0.2, random_state=42)

In [9]:
# define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=[4], activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [10]:
# compile the model
model.compile(optimizer='adam', loss='mse')

In [11]:
# train the model
model.fit(train_features, train_labels, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
52/52 [==============================] - 1s 5ms/step - loss: 1228.7699 - val_loss: 1217.2827
Epoch 2/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4652 - val_loss: 1217.2827
Epoch 3/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4653 - val_loss: 1217.2827
Epoch 4/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4650 - val_loss: 1217.2827
Epoch 5/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4651 - val_loss: 1217.2827
Epoch 6/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4651 - val_loss: 1217.2827
Epoch 7/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4650 - val_loss: 1217.2827
Epoch 8/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4650 - val_loss: 1217.2827
Epoch 9/100
52/52 [==============================] - 0s 3ms/step - loss: 1228.4652 - val_loss: 1217.2827
Epoch 10/100
52/52 [==============================] - 0

In [12]:
# evaluate the model on test data
test_predictions = model.predict(test_features)

17/17 [==============================] - 0s 1ms/step


In [13]:
# Define the contest number and extract the last 6 winning numbers
contest_number = data['Conc'].max() + 1 # assuming the next contest number is one greater than the max contest number in the dataset
last_six_numbers = data.loc[data['Conc'] == contest_number - 1, ['NR1', 'NR2', 'NR3', 'NR4', 'NR5', 'NR6']].values.ravel()

# Generate next 6 numbers
next_six_numbers = []
while len(next_six_numbers) < 6:
    num = np.random.randint(1, 61) # assuming Mega Sena has 60 balls
    if num not in last_six_numbers and num not in next_six_numbers:
        next_six_numbers.append(num)

print(f"The next 6 numbers for contest {contest_number} of Mega Sena are: {next_six_numbers}")


The next 6 numbers for contest 2602 of Mega Sena are: [9, 39, 49, 42, 50, 57]
